In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
read_df=spark.read.options(Delimiter=' ',header='True',inferschema=True).csv('/content/ops.txt')
read_df.show()

+---------+----+-----------+---+----+-----+----+
|  Country|Name|CountryCode|_c3|Year|Value| _c6|
+---------+----+-----------+---+----+-----+----+
|    India|  91|       2024|  1|null| null|null|
|Australia|  12|       2024|  2|null| null|null|
|  England|  13|       2024|  3|null| null|null|
|      USA|  14|       2024|  4|null| null|null|
|      USA|  14|       2024|  4|null| null|null|
+---------+----+-----------+---+----+-----+----+



In [ ]:
read_df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Name: integer (nullable = true)
 |-- CountryCode: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- _c6: string (nullable = true)



In [ ]:
read_df.columns

['Country', 'Name', 'CountryCode', '_c3', 'Year', 'Value', '_c6']

In [ ]:
read_df.count()

4

In [ ]:
read_df.show(2)

+---------+----+-----------+---+----+-----+----+
|  Country|Name|CountryCode|_c3|Year|Value| _c6|
+---------+----+-----------+---+----+-----+----+
|    India|  91|       2024|  1|null| null|null|
|Australia|  12|       2024|  2|null| null|null|
+---------+----+-----------+---+----+-----+----+
only showing top 2 rows



In [ ]:
#duplicate row
df=spark.read.options(Delimiter=' ',header='True',inferschema=True).csv('/content/ops.txt')
df.show()

+-----------+-----------+----+-----+
|CountryName|CountryCode|Year|Value|
+-----------+-----------+----+-----+
|      India|         91|2024|    1|
|  Australia|         12|2024|    2|
|    England|         13|2024|    3|
|        USA|         14|2024|    4|
|        USA|         14|2024|    4|
+-----------+-----------+----+-----+



In [ ]:
df.dropDuplicates().show()

+-----------+-----------+----+-----+
|CountryName|CountryCode|Year|Value|
+-----------+-----------+----+-----+
|      India|         91|2024|    1|
|    England|         13|2024|    3|
|  Australia|         12|2024|    2|
|        USA|         14|2024|    4|
+-----------+-----------+----+-----+



In [ ]:
#exceptAll
df.exceptAll(df.distinct()).show()

+-----------+-----------+----+-----+
|CountryName|CountryCode|Year|Value|
+-----------+-----------+----+-----+
|        USA|         14|2024|    4|
+-----------+-----------+----+-----+



In [ ]:
#Filter
from pyspark.sql.functions import col
df.filter(col("CountryName")=='India').show()


+-----------+-----------+----+-----+
|CountryName|CountryCode|Year|Value|
+-----------+-----------+----+-----+
|      India|         91|2024|    1|
+-----------+-----------+----+-----+



In [ ]:
#sorting and Order
df.select("CountryCode","value").show()

+-----------+-----+
|CountryCode|value|
+-----------+-----+
|         91|    1|
|         12|    2|
|         13|    3|
|         14|    4|
|         14|    4|
+-----------+-----+



In [ ]:
df.select('CountryName','value').orderBy('value',ascending=False).show()

+-----------+-----+
|CountryName|value|
+-----------+-----+
|        USA|    4|
|        USA|    4|
|    England|    3|
|  Australia|    2|
|      India|    1|
+-----------+-----+



In [ ]:
#groupby
# do filter,groupby,aggreation all in one function
from pyspark.sql.functions import col,sum
result = df.filter(col("Value")<3).groupBy("CountryName","Year").agg(sum("Value").alias("TotalValue"))

result.show()


+-----------+----+----------+
|CountryName|Year|TotalValue|
+-----------+----+----------+
|      India|2024|         1|
|  Australia|2024|         2|
+-----------+----+----------+



In [ ]:
#changing dataframe name (Alias)


In [ ]:
#change multiple column names
rename = {"CountryName" : "Country", "CountryCode" : "Code"}
df.toDF(*[rename.get(col, col) for col in df.columns]).show()

+---------+----+----+-----+
|  Country|Code|Year|Value|
+---------+----+----+-----+
|    India|  91|2024|    1|
|Australia|  12|2024|    2|
|  England|  13|2024|    3|
|      USA|  14|2024|    4|
|      USA|  14|2024|    4|
+---------+----+----+-----+



In [ ]:
#partition and re-partition and shuffeling